<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model to detect dogs.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [13]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

Import the required packages.

In [14]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import files

from mediapipe_model_maker import object_detector

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'   dist	     'My Drive'		  vehiclecount
 data.zip	    freedomtech.zip   testgooglemachine   yolov4-tiny


In [16]:
!unzip /mydrive/freedomtech.zip


Archive:  /mydrive/freedomtech.zip
   creating: freedomtech/
   creating: freedomtech/freedomtech/
   creating: freedomtech/freedomtech/train/
   creating: freedomtech/freedomtech/train/Annotations/
  inflating: freedomtech/freedomtech/train/Annotations/helmet_10.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_11.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_12.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_13.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_14.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_15.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_16.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_17.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_18.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_19.xml  
  inflating: freedomtech/freedomtech/train/Annotations/helmet_20.xml  
  inflating: freedom

### Prepare the dataset

This dataset contains about 130 images of 3 different dogs: a Golden Retriever, a Labrador Retriever mix, and a brindle-colored Formosan Mountain Dog. This is an example image from the dataset.

![aci.jpg](https://storage.googleapis.com/tflite-task-tfjs/aci.jpeg)

We start with downloading the dataset.

In [ ]:
!wget https://storage.googleapis.com/mediapipe-assets/dogs2.zip --no-check-certificate
!unzip dogs2.zip
train_dataset_path = "dogs/train"
validation_dataset_path = "dogs/validate"

In [17]:
train_dataset_path = "/content/freedomtech/freedomtech/train"
validation_dataset_path = "/content/freedomtech/freedomtech/validate"

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [20]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    '/content/freedomtech/freedomtech/train',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    '/content/freedomtech/freedomtech/validate',
    cache_dir="/tmp/od_data/validatation")

### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [21]:
hparams = object_detector.HParams(batch_size=8, learning_rate=0.3, epochs=50, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i256
Model: "retina_net_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   149056    
                             , '4': (Non

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


7/7 [==============================] - 57s 1s/step - total_loss: 26.3803 - cls_loss: 25.6922 - box_loss: 0.0127 - model_loss: 26.3253 - val_total_loss: 2.0039 - val_cls_loss: 1.2378 - val_box_loss: 0.0142 - val_model_loss: 1.9489
Epoch 2/50
7/7 [==============================] - 7s 949ms/step - total_loss: 1.7040 - cls_loss: 1.0882 - box_loss: 0.0112 - model_loss: 1.6491 - val_total_loss: 1.8798 - val_cls_loss: 1.1571 - val_box_loss: 0.0134 - val_model_loss: 1.8249
Epoch 3/50
7/7 [==============================] - 3s 450ms/step - total_loss: 1.3674 - cls_loss: 0.9020 - box_loss: 0.0082 - model_loss: 1.3124 - val_total_loss: 1.6848 - val_cls_loss: 1.0516 - val_box_loss: 0.0116 - val_model_loss: 1.6299
Epoch 4/50
7/7 [==============================] - 2s 319ms/step - total_loss: 1.1476 - cls_loss: 0.7835 - box_loss: 0.0062 - model_loss: 1.0926 - val_total_loss: 1.5244 - val_cls_loss: 0.9414 - val_box_loss: 0.0106 - val_model_loss: 1.4694
Epoch 5/50
7/7 [==============================] - 

### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 27 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [22]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

14/14 [==============================] - 1s 43ms/step - total_loss: 0.9175 - cls_loss: 0.4844 - box_loss: 0.0076 - model_loss: 0.8626
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.97s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.383
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.062
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.184
 Average Recall     (AR) @[

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [23]:
model.export_model('helmet.tflite')
!ls exported_model
files.download('exported_model/helmet.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


dogs.tflite  float_ckpt.data-00000-of-00001  float_ckpt.index  helmet.tflite  metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>